<a href="https://colab.research.google.com/github/anokhina-rgb/Multilingual-Corpus-for-EU-Studies/blob/main/mp3splitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1️⃣ Install required packages
!pip install -q pydub whisper-timestamped torch torchaudio

# 2️⃣ Mount Google Drive (optional)
from google.colab import drive
drive.mount('/content/drive')

# 3️⃣ Import libraries
import os
from pydub import AudioSegment
import whisper_timestamped as whisper
import torch

# 4️⃣ User configuration
# Change paths below: either local upload or Google Drive folder
input_folder = "/content/drive/MyDrive/mp3_input"   # <-- your MP3 folder
output_folder = "/content/drive/MyDrive/mp3_output" # <-- processed files will go here
pause_seconds = 10  # pause between segments
model_size = "base" # whisper model: tiny/base/small/medium
device = "cuda" if torch.cuda.is_available() else "cpu"

os.makedirs(output_folder, exist_ok=True)
pause_ms = pause_seconds * 1000

# 5️⃣ Load Whisper model
print("Loading Whisper model...")
model = whisper.load_model(model_size, device=device)
print("Model loaded.\n")

# 6️⃣ Process all MP3 files
files = [f for f in os.listdir(input_folder) if f.lower().endswith(".mp3")]
for idx, file_name in enumerate(files, 1):
    print(f"Processing {file_name} ({idx}/{len(files)})...")
    in_path = os.path.join(input_folder, file_name)
    out_path = os.path.join(output_folder, os.path.splitext(file_name)[0] + "__pause.mp3")

    original = AudioSegment.from_file(in_path)
    audio = whisper.load_audio(in_path)
    result = whisper.transcribe(model, audio, language=None)
    silence = AudioSegment.silent(duration=pause_ms)
    final_audio = AudioSegment.empty()

    segments = result.get("segments", [])
    if not segments:
        final_audio = original
    else:
        for i, seg in enumerate(segments):
            start_ms = int(seg["start"] * 1000)
            end_ms = int(seg["end"] * 1000)
            chunk = original[start_ms:end_ms]
            final_audio += chunk
            if i != len(segments) - 1:
                final_audio += silence
            print(f"  Segment {i+1}/{len(segments)} processed")

    final_audio.export(out_path, format="mp3")
    print(f"Saved: {out_path}\n")

print("All files processed!")


Mounted at /content/drive
Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.

Loading Whisper model...


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 100MiB/s]


Model loaded.



FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/mp3_input'